In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import os
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

/home/talat/anaconda/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
mnist = input_data.read_data_sets("../data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ../data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ../data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflo

In [3]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fe9e86e1278>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fe9da596860>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fe9da596fd0>)

In [4]:
# Hyperparameters
learning_rate = 0.01
training_iteration = 30
batch_size = 100
display_step = 2

In [5]:
# TF Graph Input
x = tf.placeholder('float', [None, 784]) # Input size 28 x 28 images
y = tf.placeholder('float', [None, 10]) # -> Class Labels

In [6]:
# Creating Model

# Model Weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

with tf.name_scope('Wx_b') as scope:
    # Linear Model
    model = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
    
# Add summary ops to collect data - just for visualization purpose
w_h = tf.summary.histogram('weights', W)
b_h = tf.summary.histogram('biases', b)


with tf.name_scope('cost_function') as scope:
    cost_function = -tf.reduce_sum(y * tf.log(model))
    tf.summary.scalar('cost_function', cost_function)
    
with tf.name_scope('train') as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)
    
    
# Initializing the variables
init = tf.global_variables_initializer()

# Merge all summaries into a single operator
merged_summary_op = tf.summary.merge_all()

In [8]:
with tf.Session() as sess:
    sess.run(init)
    
    summary_writer = tf.summary.FileWriter('../log/', graph_def=sess.graph_def)
    
    # Training Cycle
    for iteration in range(training_iteration):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            #Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y:batch_ys})
            
            # Compute the average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y:batch_ys})/total_batch
            
            # Write logs for each iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y:batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + i)
            
        # Display logs per iteration step
        if iteration % display_step == 0:
            print('Iteration: ', str(iteration+1), 'cost= ', str(avg_cost))
            
    print('Training completed')
    
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y,1))
    
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(predictions, 'float'))
    print('Accuracy= ', accuracy.eval({x:mnist.test.images, y: mnist.test.labels}))

Iteration:  1 cost=  30.131518802642844
Iteration:  3 cost=  21.05824248227206
Iteration:  5 cost=  20.09777729728005
Iteration:  7 cost=  19.71756788947365
Iteration:  9 cost=  19.347129501862963
Iteration:  11 cost=  19.099023107181896
Iteration:  13 cost=  18.938719470284173
Iteration:  15 cost=  18.814514350891123
Iteration:  17 cost=  18.675291718569667
Iteration:  19 cost=  18.581753168973076
Iteration:  21 cost=  18.42562402595174
Iteration:  23 cost=  18.34003214836121
Iteration:  25 cost=  18.206345844268796
Iteration:  27 cost=  18.22784722718326
Iteration:  29 cost=  18.17819847193633
Training completed
Accuracy=  0.9227
